# Tool recommendation with Attention network 
## (Gated recurrent units Attention neural network with weighted cross-entropy loss)

In [1]:
import numpy as np
import json
import warnings
import operator

import tensorflow as tf
from keras import backend as K
import h5py

warnings.filterwarnings("ignore")


class BahdanauAttention(tf.keras.layers.Layer):
    def __init__(self, units, **kwargs):
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)
        self.units = units
        super(BahdanauAttention, self).__init__(**kwargs)

    def get_config(self):
        return super().get_config().copy()

    def call(self, query, values):
        # query hidden state shape == (batch_size, hidden size)
        # query_with_time_axis shape == (batch_size, 1, hidden size)
        # values shape == (batch_size, max_len, hidden size)
        # we are doing this to broadcast addition along the time axis to calculate the score
        query_with_time_axis = tf.expand_dims(query, 1)

        # score shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying score to self.V
        # the shape of the tensor before applying self.V is (batch_size, max_length, units)
        score = self.V(tf.nn.tanh(self.W1(query_with_time_axis) + self.W2(values)))

        # attention_weights shape == (batch_size, max_length, 1)
        attention_weights = tf.nn.softmax(score, axis=1)

        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights


class ToolPredictionAttentionModel():
  
    def __init__(self, parameters):
        self.embedding_size = int(parameters["embedding_size"])
        self.gru_units = int(parameters["gru_units"])
        self.max_len = parameters["max_len"]
        self.dimensions = parameters["dimensions"]
        self.learning_rate = parameters["learning_rate"]
        self.class_weights = parameters["class_weights"]
        self.spatial_dropout = parameters["spatial_dropout"]
        self.recurrent_dropout = parameters["recurrent_dropout"]
        self.dropout = parameters["dropout"]
        
    def weighted_loss(self, class_weights):
        """
        Create a weighted loss function. Penalise the misclassification
        of classes more with the higher usage
        """
        weight_values = list(class_weights.values())

        def weighted_binary_crossentropy(y_true, y_pred):
            # add another dimension to compute dot product
            expanded_weights = tf.keras.backend.expand_dims(weight_values, axis=-1)
            return tf.keras.backend.dot(tf.keras.backend.binary_crossentropy(y_true, y_pred), expanded_weights)
        return weighted_binary_crossentropy

    def create_model(self):
        sequence_input = tf.keras.layers.Input(shape=(self.max_len,), dtype='int32')
        embedded_sequences = tf.keras.layers.Embedding(self.dimensions, self.embedding_size, input_length=self.max_len, mask_zero=True)(sequence_input)
        embedded_sequences = tf.keras.layers.SpatialDropout1D(self.spatial_dropout)(embedded_sequences)
        gru_1 = tf.keras.layers.GRU(self.gru_units,
            return_sequences=True,
            return_state=False,
            activation='elu',
            recurrent_dropout=self.recurrent_dropout
        )
        gru_2 = tf.keras.layers.GRU(self.gru_units,
            return_sequences=True,
            return_state=True,
            activation='elu',
            recurrent_dropout=self.recurrent_dropout
        )
        gru_output = gru_1(embedded_sequences)
        gru_output = tf.keras.layers.Dropout(self.dropout)(gru_output)
        gru_output, gru_hidden = gru_2(gru_output)
        attention = BahdanauAttention(self.gru_units)
        context_vector, attention_weights = attention(gru_hidden, gru_output)
        dropout = tf.keras.layers.Dropout(self.dropout)(context_vector)
        output = tf.keras.layers.Dense(self.dimensions, activation='sigmoid')(dropout)
        model = tf.keras.Model(inputs=sequence_input, outputs=output)
        model.compile(
            optimizer=tf.keras.optimizers.RMSprop(learning_rate=self.learning_rate),
            loss=self.weighted_loss(self.class_weights),
        )
        return model


def load_model(model_path):
    trained_model = h5py.File(model_path, 'r')
    dictionary = json.loads(trained_model.get('data_dictionary').value)
    best_parameters = json.loads(trained_model.get('parameters').value)
    compatible_tools = json.loads(trained_model.get('compatible_tools').value)
    reverse_dictionary = dict((str(v), k) for k, v in dictionary.items())
    new_model = ToolPredictionAttentionModel(best_parameters).create_model()
    new_model.load_weights(model_path)
    return dictionary, reverse_dictionary, best_parameters, compatible_tools, new_model


model_path = "data/tool_recommendation_attention_model.hdf5"
dictionary, reverse_dictionary, best_parameters, compatible_tools, new_model = load_model(model_path)

Using TensorFlow backend.


## Unpack trained model for prediction

In [2]:
print(reverse_dictionary)

{'1': 'PeptideIndexer', '2': 'picard_ReorderSam', '3': 'openms_id_file_converter', '4': 'interproscan', '5': 'coords2clnt.py', '6': 'join1', '7': 'openms_protein_quantifier', '8': 'bedtools_coveragebed', '9': 'seq_filter_by_id', '10': 'FileMerger', '11': 'bed2gff1', '12': 'trim_galore', '13': 'ConsensusID', '14': 'vt_normalize', '15': 'picard_NormalizeFasta', '16': 'bam_to_sam', '17': 'cshl_word_list_grep', '18': 'Extract genomic DNA 1', '19': 'MSGFPlusAdapter', '20': 'DatamashOps', '21': 'ncbi_tblastn_wrapper', '22': 'p_clip_peaks', '23': 'vcffilter', '24': 'EMBOSS: shuffleseq87', '25': 'vcftools_isec', '26': 'fasta2tab', '27': 'bedtools_sortbed', '28': 'IDFilter', '29': 'bismark_bowtie2', '30': 'Add_a_column1', '31': 'r_correlation_matrix', '32': 'Count1', '33': 'EMBOSS: fuzztran39', '34': 'deeptools_computeMatrix', '35': 'HighResPrecursorMassCorrector', '36': 'tp_sed_tool', '37': 'deeptools_bamCoverage', '38': 'cshl_fastx_trimmer', '39': 'secretbt2test', '40': 'CONVERTER_bedgraph_to

In [3]:
def compute_recommendations(tool_sequence, labels, topk=20, max_seq_len=25):
    tl_seq = tool_sequence.split(",")
    last_tool_name = reverse_dictionary[str(tl_seq[-1])]
    sample = np.zeros(max_seq_len)
    for idx, tool_id in enumerate(tl_seq):
        sample[idx] = int(tool_id)
    sample_reshaped = np.reshape(sample, (1, max_seq_len))
    tool_sequence_names = [reverse_dictionary[str(tool_pos)] for tool_pos in tool_sequence.split(",")]
    # predict next tools for a test path
    prediction = new_model.predict(sample_reshaped, verbose=0)
    class_weights = best_parameters["class_weights"]
    weight_val = list(class_weights.values())
    weight_val = np.reshape(weight_val, (len(weight_val),))
    prediction = np.reshape(prediction, (prediction.shape[1],))
    #prediction = prediction * weight_val
    prediction = prediction / float(np.max(prediction))
    prediction_pos = np.argsort(prediction, axis=-1)
    # get topk prediction
    topk_prediction_pos = prediction_pos[-topk:]
    topk_prediction_pos = [item for item in topk_prediction_pos if item != 0]
    topk_prediction_val = [int(prediction[pos] * 100) for pos in topk_prediction_pos]
    # read tool names using reverse dictionary
    pred_tool_ids = [reverse_dictionary[str(tool_pos)] for tool_pos in topk_prediction_pos]
    pred_tool_ids_sorted = dict()
    for (tool_pos, tool_pred_val) in zip(topk_prediction_pos, topk_prediction_val):
        tool_name = reverse_dictionary[str(tool_pos)]
        pred_tool_ids_sorted[tool_name] = tool_pred_val
    pred_tool_ids_sorted = dict(sorted(pred_tool_ids_sorted.items(), key=lambda kv: kv[1], reverse=True))
    ids_tools = dict()
    keys = list(pred_tool_ids_sorted.keys())
    tool_seq_name = ",".join(tool_sequence_names)
    print("Current tool sequence: ")
    print()
    print(tool_seq_name)
    print()
    print("Recommended tools for the tool sequence '%s' with their scores in decreasing order:" % tool_seq_name)
    print()
    for i in pred_tool_ids_sorted:
        print(i + "(" + str(pred_tool_ids_sorted[i]) + "%)")
    for key in pred_tool_ids_sorted:
        ids_tools[key] = dictionary[key]
    print()
    print("Tool ids:")
    print()
    for i in ids_tools:
        print(i + "(" + str(ids_tools[i]) + ")")

## Indices of tools

In [8]:
topk = 20 # set the maximum number of recommendations
tool_seq = "137" # give tools ids in a sequence and see the recommendations. To know all the tool ids, 
                     # please print the variable 'reverse_dictionary'
compute_recommendations(tool_seq, "", topk)

Current tool sequence: 

Cut1

Recommended tools for the tool sequence 'Cut1' with their scores in decreasing order:

comp1(100%)
CONVERTER_interval_to_bed_0(97%)
cat1(96%)
Remove beginning1(95%)
tp_easyjoin_tool(94%)
tp_sort_header_tool(94%)
methtools_destrand(93%)
Filter1(92%)
wc_gnu(92%)
Add_a_column1(90%)
cshl_uniq_tool(90%)
join1(89%)
sort1(88%)
Grep1(87%)
cshl_find_and_replace(86%)
methtools_dmr(86%)
Paste1(85%)
tp_replace_in_column(85%)
bg_uniq(85%)
gops_intersect_1(85%)

Tool ids:

comp1(140)
CONVERTER_interval_to_bed_0(227)
cat1(295)
Remove beginning1(75)
tp_easyjoin_tool(262)
tp_sort_header_tool(186)
methtools_destrand(131)
Filter1(50)
wc_gnu(256)
Add_a_column1(30)
cshl_uniq_tool(176)
join1(6)
sort1(65)
Grep1(166)
cshl_find_and_replace(250)
methtools_dmr(275)
Paste1(99)
tp_replace_in_column(77)
bg_uniq(286)
gops_intersect_1(185)


## Recommended tools

In [5]:
print(new_model.summary())

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 25)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 25, 192)           59520     
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 25, 192)           0         
_________________________________________________________________
gru (GRU)                    (None, 25, 212)           258216    
_________________________________________________________________
dropout (Dropout)            (None, 25, 212)           0         
_________________________________________________________________
gru_1 (GRU)                  [(None, 25, 212), (None,  270936    
_________________________________________________________________
bahdanau_attention (Bahdanau ((None, 212), (None, 25,  90525 

In [6]:
print(best_parameters)

{'embedding_size': 192, 'gru_units': 212, 'spatial_dropout': 0.25, 'dropout': 0.30000000000000004, 'recurrent_dropout': 0.15000000000000002, 'learning_rate': 0.013046726432017543, 'batch_size': 32, 'max_len': 25, 'dimensions': 310, 'class_weights': {'0': 0.0, '1': 4.993573370845278, '2': 0.09531073434962559, '3': 0.0, '4': 3.5483850691681953, '5': 0.6600210747001517, '6': 5.785395391705502, '7': 0.0, '8': 4.206458375495826, '9': 2.651965442452904, '10': 0.3668217386599276, '11': 1.4713061761506316, '12': 6.982862751468942, '13': 3.0032081216359656, '14': 0.2527023535557542, '15': 2.4336133554004498, '16': 0.0, '17': 0.07189469179761168, '18': 4.146814615224232, '19': 0.0, '20': 0.061612021235966506, '21': 2.285273209338971, '22': 0.09359915319644818, '23': 0.07250647687303469, '24': 0.06263988862502112, '25': 0.5534520941998734, '26': 6.190901061884645, '27': 4.490221318991273, '28': 4.8283283891949615, '29': 2.36302973908641, '30': 5.888814776503156, '31': 0.0, '32': 3.747675549429567